## Import

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
from huggingface_hub import notebook_login

notebook_login()

In [ ]:
# import locale
# locale.getpreferredencoding = lambda: "UTF-8"
!pip install sentence_transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 156.5/156.5 kB 5.1 MB/s eta 0:00:00


In [ ]:
!pip install -q -U bitsandbytes
!pip install -q -U git+https://github.com/huggingface/transformers.git
!pip install -q -U git+https://github.com/huggingface/peft.git
!pip install -q -U git+https://github.com/huggingface/accelerate.git
!pip install -q datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.0/105.0 MB 16.3 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 280.0/280.0 kB 7.1 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 510.5/510.5 kB 8.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 11.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 8.7 MB/s eta 0:00:00


In [ ]:
!pip install -U torch tokenizers safetensors

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 755.5/755.5 MB 2.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 62.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 65.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 87.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 731.7/731.7 MB 2.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 3.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.6/121.6 MB 14.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.5/56.5 MB 30.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.2/124.2 MB 7.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 196.0/196.0 MB 4.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 166.0/166.0 MB 10.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.1/99.1 kB 14.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━

In [ ]:
import pandas as pd
import numpy as np
import torch
from transformers import  AdamW
from tqdm import tqdm

In [ ]:
# CUDA 사용 가능 여부 확인
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

Using device: cuda


In [ ]:
from peft import (
    get_peft_config,
    get_peft_model,
    get_peft_model_state_dict,
    set_peft_model_state_dict,
    LoraConfig,
    PeftType,
    PrefixTuningConfig,
    PromptEncoderConfig,
    PeftModel,
    PeftConfig,
)

peft_type = PeftType.LORA
peft_config = LoraConfig(
    r=8,
    lora_alpha=32,
    target_modules=[
    "q_proj",
    "up_proj",
    "o_proj",
    "k_proj",
    "down_proj",
    "gate_proj",
    "v_proj"],
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM"
)


## Data Preprocessing

In [ ]:
import pandas as pd
import numpy as np
import torch
import transformers
import bitsandbytes as bnb
import os
# import wandb

from transformers import PreTrainedTokenizerFast, AdamW, AutoModelForCausalLM, BitsAndBytesConfig, AutoTokenizer
from peft import prepare_model_for_kbit_training, LoraConfig, get_peft_model
from tqdm import tqdm

#os.environ["TOKENIZERS_PARALLELISM"] = "true"
#torch.backends.cuda.matmul.allow_tf32=True
#torch.set_float32_matmul_precision('medium')
#torch.backends.cudnn.benchmark = True

In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig

model_id = "LDCC/LDCC-SOLAR-10.7B"
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16
)

tokenizer = AutoTokenizer.from_pretrained(model_id, trust_remote_code=True,eos_token='</s>')
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"
model = AutoModelForCausalLM.from_pretrained(model_id, quantization_config=bnb_config, device_map={"":0})
model.gradient_checkpointing_enable()
model = prepare_model_for_kbit_training(model)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/2.81M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/4.77M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/557 [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


...

model-00003-of-00005.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00004-of-00005.safetensors:   0%|          | 0.00/4.92G [00:00<?, ?B/s]

model-00005-of-00005.safetensors:   0%|          | 0.00/1.94G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/5 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/136 [00:00<?, ?B/s]

In [ ]:
# compute_dtype = getattr(torch, "float16")
# bnb_config = BitsAndBytesConfig(
#         load_in_4bit=True,
#         bnb_4bit_quant_type="nf4",
#         bnb_4bit_compute_dtype=compute_dtype,
#         bnb_4bit_use_double_quant=True,
# )
# model = AutoModelForCausalLM.from_pretrained('emaeon/solar-hansol-pretrain-merge',quantization_config=bnb_config, device_map={"": 0}).to(device)

In [ ]:
# model_name = "LDCC/LDCC-SOLAR-10.7B"
# adapter = "emaeon/solar-hansol-pretrain"
# compute_dtype = getattr(torch, "float16")
# bnb_config = BitsAndBytesConfig(
#         load_in_4bit=True,
#         bnb_4bit_quant_type="nf4",
#         bnb_4bit_compute_dtype=compute_dtype,
#         bnb_4bit_use_double_quant=True,
# )
# model = AutoModelForCausalLM.from_pretrained(
#           model_name, quantization_config=bnb_config, device_map={"": 0}
# )
# model = PeftModel.from_pretrained(model, adapter)

In [ ]:
# import torch
# from peft import PeftModel, PeftConfig
# from transformers import AutoModelForCausalLM, AutoTokenizer

# peft_model_id = "emaeon/solar-hansol-finetuning"
# config = PeftConfig.from_pretrained(peft_model_id)
# inference_model = PeftModel.from_pretrained(model, peft_model_id)

# inference_model.to(device)
# # inference_model.eval()

# inference_model = inference_model.merge_and_unload()

In [ ]:
from peft import prepare_model_for_kbit_training

model.gradient_checkpointing_enable()
model = prepare_model_for_kbit_training(model)
model = get_peft_model(model, peft_config).to(device)

model.print_trainable_parameters()
model


trainable params: 31,457,280 || all params: 10,894,053,376 || trainable%: 0.28875643357229686


PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): LlamaForCausalLM(
      (model): LlamaModel(
        (embed_tokens): Embedding(48000, 4096, padding_idx=2)
        (layers): ModuleList(
          (0-47): 48 x LlamaDecoderLayer(
            (self_attn): LlamaSdpaAttention(
              (q_proj): lora.Linear4bit(
                (base_layer): Linear4bit(in_features=4096, out_features=4096, bias=False)
                (lora_dropout): ModuleDict(
                  (default): Dropout(p=0.05, inplace=False)
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=4096, out_features=8, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=8, out_features=4096, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embedding_B): ParameterDict()
              )
              (k_proj): lora.Linear4bit(
                (base_la

In [ ]:
# conversation = [ {'role': 'user', 'content': 'Hello?'} ]

# prompt = tokenizer.apply_chat_template(conversation, tokenize=False, add_generation_prompt=True)

# inputs = tokenizer(prompt, return_tensors="pt").to(model.device)
# outputs = model.generate(**inputs, use_cache=True, max_length=4096)
# output_text = tokenizer.decode(outputs[0])
# print(output_text)

<s> ### User:
Hello?

### Assistant:
Hello, how can I assist you today?</s>


In [ ]:
# import pandas as pd
# from tqdm import tqdm

# # 데이터 로드
# data = pd.read_csv('/content/drive/MyDrive/DACON_도배하자/v2train.csv')

# # 데이터 포맷팅 및 토크나이징
# formatted_data = []
# input_texts = []
# for _, row in tqdm(data.iterrows()):
#     for q_col in ['질문_1', '질문_2']:
#         for a_col in ['답변_1', '답변_2', '답변_3', '답변_4', '답변_5']:
#             # 질문과 답변 쌍을 </s> token으로 연결
#             input_text = '###질문:' + row[q_col] + '\n\n###답변:'+ row[a_col]+tokenizer.eos_token
#             # input_text = '<s> ### User:' + row[q_col] + '\n\n### Assistant:\n'+ row[a_col]+tokenizer.eos_token
#             input_texts.append(input_text)

# # 데이터프레임 생성
# df = pd.DataFrame({'text': input_texts})
# print(df.head())


644it [00:00, 8564.01it/s]

                                                text
0  ###질문:면진장치가 뭐야?\n\n###답변:면진장치란 지반에서 오는 진동 에너지를...
1  ###질문:면진장치가 뭐야?\n\n###답변:면진장치란 건물의 지반에서 발생하는 진...
2  ###질문:면진장치가 뭐야?\n\n###답변:면진장치란 지반으로부터 발생하는 진동 ...
3  ###질문:면진장치가 뭐야?\n\n###답변:면진장치는 건물의 지반으로부터 오는 진...
4  ###질문:면진장치가 뭐야?\n\n###답변:면진장치는 건물에 오는 지반 진동의 영...


In [ ]:
df = pd.read_csv('/content/drive/MyDrive/DACON_도배하자/v2_train(모든수정완료).csv')

In [ ]:
max_length = df['text'].str.len().max()
print("최대 길이:", max_length)

최대 길이: 1260


In [ ]:
max_row_lengths = df.apply(lambda row: max(row.str.len()), axis=1)

# 최대 문자열 길이 출력
print("각 행의 최대 문자열 길이:")
print(max_row_lengths)
print("최대 문자열 길이:", max_row_lengths.max())
length_freq = {}

# 각 행의 최대 길이를 기반으로 딕셔너리에 빈도 업데이트
for length in max_row_lengths:
    if length in length_freq:
        length_freq[length] += 1
    else:
        length_freq[length] = 1

# print(length_freq)
# 빈도순으로 정렬
sorted_length_freq = dict(sorted(length_freq.items(), key=lambda x: x[1], reverse=True))

print(sorted_length_freq)

각 행의 최대 문자열 길이:
0      NaN
1      NaN
2      NaN
3      NaN
4      NaN
        ..
7462   NaN
7463   NaN
7464   NaN
7465   NaN
7466   NaN
Length: 7467, dtype: float64
최대 문자열 길이: nan
{nan: 1, nan: 1, nan: 1, nan: 1, nan: 1, nan: 1, nan: 1, nan: 1, nan: 1, nan: 1, nan: 1, nan: 1, nan: 1, nan: 1, nan: 1, nan: 1, nan: 1, nan: 1, nan: 1, nan: 1, nan: 1, nan: 1, nan: 1, nan: 1, nan: 1, nan: 1, nan: 1, nan: 1, nan: 1, nan: 1, nan: 1, nan: 1, nan: 1, nan: 1, nan: 1, nan: 1, nan: 1, nan: 1, nan: 1, nan: 1, nan: 1, nan: 1, nan: 1, nan: 1, nan: 1, nan: 1, nan: 1, nan: 1, nan: 1, nan: 1, nan: 1, nan: 1, nan: 1, nan: 1, nan: 1, nan: 1, nan: 1, nan: 1, nan: 1, nan: 1, nan: 1, nan: 1, nan: 1, nan: 1, nan: 1, nan: 1, nan: 1, nan: 1, nan: 1, nan: 1, nan: 1, nan: 1, nan: 1, nan: 1, nan: 1, nan: 1, nan: 1, nan: 1, nan: 1, nan: 1, nan: 1, nan: 1, nan: 1, nan: 1, nan: 1, nan: 1, nan: 1, nan: 1, nan: 1, nan: 1, nan: 1, nan: 1, nan: 1, nan: 1, nan: 1, nan: 1, nan: 1, nan: 1, nan: 1, nan: 1, nan: 1, nan: 1, na

In [ ]:
from datasets import load_dataset
dataset = load_dataset("csv", data_files="/content/drive/MyDrive/DACON_도배하자/v2_train(모든수정완료).csv")
# testdataset = load_dataset("csv", data_files="/content/drive/MyDrive/DACON_도배하자/test.csv")

Generating train split: 0 examples [00:00, ? examples/s]

In [ ]:
# ##확인

# outputs = tokenizer(
#     dataset["train"][:2]["text"],
#     truncation=True,
#     max_length=300,
#     return_overflowing_tokens=True,
#     return_length=True,
# )

# outputs

In [ ]:
# context_length=128
# def tokenize(element):
#     outputs = tokenizer(
#         element['text'],
#         truncation=True,
#         max_length=context_length,
#         return_overflowing_tokens=True,
#         return_length=True,
#     )
#     input_batch = []
#     for length, input_ids in zip(outputs['length'], outputs['input_ids']):
#         if length == context_length:
#         # print(length)
#           input_batch.append(input_ids)
#     return {"input_ids": input_batch}


# tokenized_datasets = dataset.map(
#     tokenize, batched=True, remove_columns=dataset["train"].column_names)
# tokenized_datasets


In [ ]:
# context_length=128
def tokenize(element):
    outputs = tokenizer(
        element['text'],
        truncation=True,
        max_length=1024,
        # return_overflowing_tokens=True,
        # return_length=True,
    )
    input_batch = []
    for input_ids in outputs['input_ids']:
        # print(length)
          input_batch.append(input_ids)
    return {"input_ids": input_batch}


tokenized_datasets = dataset.map(
    tokenize, batched=True, remove_columns=dataset["train"].column_names)
tokenized_datasets


Map:   0%|          | 0/7467 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['input_ids'],
        num_rows: 7467
    })
})

## Model Fine-tuning

In [ ]:
from transformers import DataCollatorForLanguageModeling

tokenizer.pad_token = tokenizer.eos_token
data_collator = DataCollatorForLanguageModeling(tokenizer, mlm=False)


In [ ]:
# #확인용
# out = data_collator([tokenized_datasets["train"][i] for i in range(5)])
# for key in out:
#     print(f"{key} shape: {out[key].shape}")


In [ ]:
from transformers import Trainer, TrainingArguments

args = TrainingArguments(
    output_dir='v2-solar-hansol-ft',
    logging_steps=10,
    per_device_train_batch_size=2,
    # evaluation_strategy="steps",
    gradient_accumulation_steps=2,
    num_train_epochs=3,
    weight_decay=0.001,
    # warmup_steps=10,
    lr_scheduler_type="cosine",
    learning_rate=3e-4,
    save_steps=10,
    fp16=True,
    push_to_hub=True,
    optim="paged_adamw_8bit"
)


trainer = Trainer(
    model=model,
    tokenizer=tokenizer,
    args=args,
    data_collator=data_collator,
    train_dataset=tokenized_datasets["train"],
)
model.config.use_cache = False
# model.config.pretraining_tp = 1

/usr/local/lib/python3.10/dist-packages/accelerate/accelerator.py:432: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(


In [ ]:
trainer.train()

/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:460: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(


Step,Training Loss
10,1.723300
20,1.226600
30,1.261800
40,1.230700
50,1.262800
60,1.272200
70,1.151100
80,1.125600
90,1.203000
100,1.158100


/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:460: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:460: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:460: UserWarning: torch.

Step,Training Loss
10,1.723300
20,1.226600
30,1.261800
40,1.230700
50,1.262800
60,1.272200
70,1.151100
80,1.125600
90,1.203000
100,1.158100


/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:460: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:460: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:460: UserWarning: torch.

TrainOutput(global_step=5601, training_loss=0.6385609074865701, metrics={'train_runtime': 15371.7674, 'train_samples_per_second': 1.457, 'train_steps_per_second': 0.364, 'total_flos': 2.978290101708964e+17, 'train_loss': 0.6385609074865701, 'epoch': 3.0})

In [ ]:
trainer.push_to_hub()

## Model Inference

In [ ]:
# model = model.merge_and_unload()

# Save the merged model in a directory "./naive_merge/" in the safetensors format
# model.save_pretrained("./merge_model/", safe_serialization=True)

/usr/local/lib/python3.10/dist-packages/peft/tuners/lora/bnb.py:248: UserWarning: Merge lora module to 4-bit linear may get different generations due to rounding errors.
  warnings.warn(


In [ ]:
# # model = AutoModelForCausalLM.from_pretrained("./naive_merge/", device_map="auto")

# compute_dtype = getattr(torch, "float16")
# bnb_config = BitsAndBytesConfig(
#         load_in_4bit=True,
#         bnb_4bit_quant_type="nf4",
#         bnb_4bit_compute_dtype=compute_dtype,
#         bnb_4bit_use_double_quant=True,
# )
# model = AutoModelForCausalLM.from_pretrained("./merge_model/", quantization_config=bnb_config, device_map={"": 0})

In [ ]:
testdata = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/test.csv')
testdata.head()

,id,질문
0,TEST_000,"방청 페인트의 종류에는 어떤 것들이 있는지 알고 계신가요? 또한, 원목사이딩을 사용..."
1,TEST_001,도배지에 녹은 자국이 발생하는 주된 원인과 그 해결 방법은 무엇인가요?
2,TEST_002,"큐블럭의 단점을 알려주세요. 또한, 압출법 단열판을 사용하는 것의 장점은 무엇인가요?"
3,TEST_003,"철골구조를 사용하는 고층 건물에서, 단열 효과를 높이기 위한 시공 방법은 무엇이 있..."
4,TEST_004,도배지의 완전한 건조를 위해 몇 주 동안 기다려야 하나요?


In [ ]:
# #필요한 모듈을 import 합니다.
# from numba import cuda

# #이후 초기화 작업을 진행해줍니다.
# # device = cuda.get_current_device()
# # device.reset()

# import torch, gc
# # gc.collect()
# # torch.cuda.empty_cache()
# import os
# # os.environ['CUDA_LAUNCH_BLOCKING'] = "1"


In [ ]:
!nvidia-smi

Wed Feb 28 04:56:10 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla V100-SXM2-16GB           Off | 00000000:00:04.0 Off |                    0 |
| N/A   37C    P0              24W / 300W |      2MiB / 16384MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [ ]:
tokenizer.eos_token

'</s>'

In [ ]:
tokenizer.vocab_size
# len(tokenizer)

32000

In [ ]:
# 모델 평가 모드로 설정
model.eval()

# 모델이 캐시 사용 여부 설정 (출력된 경고 메시지를 없애기 위함. 추론 시 다시 활성화할 것!)
model.config.use_cache = True
def gen(x):
    # 모델에 입력으로 사용할 문장 생성
    input_text = f"###질문:{x}\n\n###답변:"
    # 토크나이징 및 모델 입력 형태로 변환
    inputs = tokenizer(
        input_text,
        return_tensors='pt',
        return_token_type_ids=False
    )

    # 생성된 텍스트(답변) 저장
    gened = model.generate(
        **inputs,
        max_length=300,
        early_stopping=True,
        do_sample=True,
        eos_token_id=2,
        top_k=1,
        temperature=0.9,
        top_p=0.9,
        repetition_penalty=1.2,
        # num_return_sequences=1
    )


    # 생성된 답변 디코딩하여 반환
    generated_text = tokenizer.decode(gened[0])
    print(generated_text)
    return generated_text

# '질문' 컬럼의 각 질문에 대해 답변 생성
preds = []
for test_question in tqdm(testdata['질문']):
    output_sequences = gen(test_question)

    # 생성된 텍스트(답변) 저장
    answer_start = output_sequences.find("\n\n###답변:") + len("\n\n###답변:")
    answer_only = output_sequences[answer_start:].strip().replace('\n', '')
    answer_only = answer_only.replace('</s>', '')
    preds.append(answer_only)


  0%|          | 0/130 [00:00<?, ?it/s]/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:453: UserWarning: `num_beams` is set to 1. However, `early_stopping` is set to `True` -- this flag is only used in beam-based generation modes. You should set `num_beams>1` or unset `early_stopping`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1482: UserWarning: You are calling .generate() with the `input_ids` being on a device type different than your model's device. `input_ids` is on cpu, whereas the model is on cuda. You may experience unexpected behaviors or slower generation. Please make sure that you have put `input_ids` to the correct device by calling for example input_ids = input_ids.to('cuda') before running `.generate()`.
  warnings.warn(
  1%|          | 1/130 [00:39<1:25:28, 39.76s/it]

<s> ###질문:방청 페인트의  종류에는  어떤  것들이  있는 지 알고 계신가요 ? 또한 , 원목사 이딩을 사용하는  것에 어떤  단점이  있을까요 ?

###답변:방청 페인트는  여러 가지  종류 가 있습니다 . 그중에서  광 명단은  주로  사용되며 , 이는  방청 효과 와 함께  색상을  제공합니다 . 다만, 광 명단이  비싸다는  점을  염두에  두어야  합니다 . 그리고  다른  종류로는  산화철이나  역청질과 같은  특수 소재 를 사용하여  제작 된 페인트가  있으며 , 각각의 장단점과  적용 분야 가 다르므로  상황 에 맞 게 선택하는  것이  중요합니다 . 특히  최근에는  환경  친 화적인  재료 나 기술 을 활용한 새로운  종류 의 방청 페인트가  개발되고  있습니다 . 이에  따라  상황 에 따라  적합한 페인트를  선택하고  사용하는  것이  좋습니다 . 그러나  무엇보다  페인트를  칠하기  전에  좋은  표


/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:453: UserWarning: `num_beams` is set to 1. However, `early_stopping` is set to `True` -- this flag is only used in beam-based generation modes. You should set `num_beams>1` or unset `early_stopping`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1482: UserWarning: You are calling .generate() with the `input_ids` being on a device type different than your model's device. `input_ids` is on cpu, whereas the model is on cuda. You may experience unexpected behaviors or slower generation. Please make sure that you have put `input_ids` to the correct device by calling for example input_ids = input_ids.to('cuda') before running `.generate()`.
  warnings.warn(
  2%|▏         | 2/130 [01:23<1:30:06, 42.24s/it]

<s> ###질문:도배지에  녹 은 자국이  발생하는  주된 원인 과 그 해결 방법 은 무엇인가요 ?

###답변:도배지 표면에  붉 은색의  녹 이 묻어나오는  현상은  일반적으로  누수가  있는  경우, 특히  수도꼭지가  누수하여  생기는  문제입니다 . 이런  상황에서는  먼 저 누수를  일으킨  부분 을 찾아내고 , 수리  작업 을 통해 문제를  해결해야  합니다 . 또한 , 도배지 표면 전체 에 녹 이 묻어나왔다 면 전문가 의 도움을  받아  적절한  조치 를 취하는  것이  좋습니다 . 만약 누수로  인한  도배지의  손상이  컸다면  교체나 보수  작업 을 진행할 때에는  전문가 와 상의하여  신중한  결정 을 내리는  것이  중요합니다 . 따라 서 누수에  따른 도배지의  오염 과 손상을  방지하기  위해서는  빠 른 시일 내에  누수 문제를  해결하고 , 필요한  경우에 는


  2%|▏         | 3/130 [01:57<1:21:15, 38.39s/it]

...

<s> ###질문:평지붕 의 누수 문제를  방지하기  위해  수성 벽 체용 탄 성 방수 도료를  사용하는  것이  어떤  장점이  있나요 ?

###답변:이 제품 은 건물의  외부나 내부에  설치되어  빗 물 등의  침투를  막아주 고, 동시에  건물을  보호해주는  역할을 합니다 . 또한 , 이 솔루션 은 비용  효율적이며  유지보수 가 간편하여  장기적으로  사용하기에  적합한 선택지가  됩니다 . 그래 서 건축 주거나  시공업체에게  권장되는  것입니다 . 만약 누수가  발생한다면  즉 시 처리해야  하며 , 시설물이  손상을  입기는  것을  예방하려면  빠르게  조치하여야  합니다 . 따라 서 이런  상황에서  효과적인  방법으로  누수를  막을  수 있는  이 제품 을 고려하시는  것을  추천드립니다 . 혹 은 추가적인  정보 를 원하시면  저렴하게  상담 을 받을  수 있는  전문가들에게  연락하실  수도  있습니다 . 감사합니다 . 여기에  댓글이나


 99%|█████████▉| 129/130 [1:12:59<00:36, 36.25s/it]

<s> ###질문:석고수정 이 발생하는  가장  큰 원인 은 무엇인가요 ? 그리고  이를  해결하는  방법 에 대해 알려주세요 ."

###답변:석고수정 의 주된 원인으로는  건물의  오래됨에  따른 물리적 변화 , 습도의  영향, 구조 적 결함 등이  있습니다 . 특히  건물이  수년 이상  사용되면서  벽이나  천장의  석고가  수정되는  경우가  많습니다 . 따라 서 정기적인  점검 과 보수 가 필요하며 , 특히  습도를  관리하고  제습기를  가동하여  실내 환기를  유지함으로써  이런  문제를  예방 할 수 있습니다 . 만약 이미  석고수정 이 발견된다면  전문가 와 상담하여  신속한 조치 를 취해야  합니다 . 또한 , 건축 적 결함에  의한 석고수정 도 주의해서  처리하여야  합니다 . 건물을  설계하고  건설 한 때에는  안전 을 고려하여  적합한 재료 와 시공  방법 을 선택하여야  합니다 . 그러므로  건축  시


100%|██████████| 130/130 [1:13:35<00:00, 33.97s/it]

<s> ###질문:카페트의  기대 수명은  얼 마나 될까요 ? 그리고  오리 지널징크의  장점에 는 무엇 이 있나요 ?

###답변:카페트는  일반적으로  6년 정도 의 사용  수명을  가지 고 있습니다 . 그러나  이는  환경 , 유지보수  상태 , 청소  주기  등에  따라  달라 질 수 있으므로  정기적인  관리 와 점검 이 필요합니다 . 또한  카페트가  낡거나  손 상된 경우, 교체하는  것이  보다  안전하고  품질 을 유지하는  방법입니다 . 만약 추가적인  정보 가 필요하다 면 전문가 의 조언을  구하는  것도  좋은  선택입니다 . 감사합니다 .   카페트에  있어 서 주요한  점은  청소 와 유지보수입니다 . 먼 저, 카페트를  구매하거나  옮기고  설치하기  전에  꼼꼼하게  청소해야  합니다 . 그 후에 도 꾸 준한 청소 를 통해 먼지나  오염물을  제거하여  카페트의  수명을  연장시켜야  합니다 . 또한 , 주기적인  유지보수 로


## Submission

In [ ]:
result = pd.DataFrame({'text':preds})
result.to_excel('/content/drive/MyDrive/Colab Notebooks/gen_result.xlsx',index=False)

In [ ]:
# Test 데이터셋의 모든 질의에 대한 답변으로부터 512 차원의 Embedding Vector 추출
# 평가를 위한 Embedding Vector 추출에 활용하는 모델은 'distiluse-base-multilingual-cased-v1' 이므로 반드시 확인해주세요.
from sentence_transformers import SentenceTransformer # SentenceTransformer Version 2.2.2

# Embedding Vector 추출에 활용할 모델(distiluse-base-multilingual-cased-v1) 불러오기
model = SentenceTransformer('distiluse-base-multilingual-cased-v1')

# 생성한 모든 응답(답변)으로부터 Embedding Vector 추출
pred_embeddings = model.encode(preds)
pred_embeddings.shape

modules.json:   0%|          | 0.00/341 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/122 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/2.47k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/556 [00:00<?, ?B/s]

...

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

2_Dense/config.json:   0%|          | 0.00/114 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/1.58M [00:00<?, ?B/s]

(130, 512)

In [ ]:

# submit = pd.read_csv('/content/drive/MyDrive/DACON_도배하자/sample_submission.csv')
submit = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/sample_submission.csv')
# 제출 양식 파일(sample_submission.csv)을 활용하여 Embedding Vector로 변환한 결과를 삽입
submit.iloc[:,1:] = pred_embeddings
submit.head()

,id,vec_0,vec_1,vec_2,vec_3,vec_4,vec_5,vec_6,vec_7,vec_8,...,vec_502,vec_503,vec_504,vec_505,vec_506,vec_507,vec_508,vec_509,vec_510,vec_511
0,TEST_000,0.029702,0.067614,0.004498,-0.028937,0.100901,0.016998,-0.042237,0.024350,0.045037,...,-0.019350,-0.039210,0.021241,-0.032649,-0.044459,-0.004297,-0.001146,-0.051986,0.002771,0.069325
1,TEST_001,-0.001891,0.018094,-0.010224,0.022551,0.098050,-0.016167,-0.011424,-0.009743,0.006543,...,-0.012079,-0.004233,0.029564,-0.006789,-0.003750,0.002754,0.019357,-0.043473,0.025181,-0.005346
2,TEST_002,0.010448,-0.014018,-0.014866,0.002995,0.145792,-0.032638,-0.028628,-0.064528,0.028069,...,-0.020428,-0.000489,0.022284,-0.018682,-0.043084,0.012404,-0.043664,-0.043362,-0.059862,0.092575
3,TEST_003,-0.004454,0.041435,0.010929,0.026986,0.092708,-0.027959,-0.081683,-0.005010,-0.006649,...,-0.039062,0.015578,0.054182,-0.030814,-0.008481,0.023496,-0.008615,-0.003119,-0.035515,0.040382
4,TEST_004,0.006720,-0.033262,-0.009605,-0.011423,0.108323,-0.033320,0.045913,0.051450,-0.011764,...,0.008158,-0.003420,0.025307,-0.004997,-0.048598,-0.017081,0.009153,0.025302,-0.012278,0.046841


In [ ]:
# 리더보드 제출을 위한 csv파일 생성
submit.to_csv('/content/drive/MyDrive/Colab Notebooks/solar_submit.csv', index=False)